# [1-2 NN]

*KU LeeDongGyu*

### Note :

This code was used during the workshop at Kyungpook National University. <br>
Also, the content of this material is what I learned while moving Pycham to Jupyter.

Reference : TensorFlow 2.1 Quick Start Guide (by Holdroyd)

### Modules

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

## 1. Example 1-2
---

In [67]:
"""
참고교재: 텐서플로로 배우는 딥러닝 (박혜정, 석경하, 심주용, 황창하 공저)

예제 1-2. UCI 기계학습 저장소에 있는 Wine 데이터를 이용하여 은닉층이 1개인 
MLP 분류 알고리즘을 작성하시오. 이때 종속변수는 첫 번째 열에 있는 Wiine의 
품종을 나타내는 class 변수이다
(자료 위치: http://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data).
"""

col_name = ["Class", "Alcohol", "Malic_acid","Ash", "Alcalinity_of_ash", 
"Magnesium","Total_phenols", "Flavanoids","Nonflavanoid_phenols",
"Proanthocyanins","Color_intensity","Hue","Diluted wines","Proline"]

In [5]:
wine = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data",header=None,
                 names=col_name,na_values=["NA", "null", ""],sep=",")
# ""도 NA취급

In [6]:
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 14 columns):
Class                   178 non-null int64
Alcohol                 178 non-null float64
Malic_acid              178 non-null float64
Ash                     178 non-null float64
Alcalinity_of_ash       178 non-null float64
Magnesium               178 non-null int64
Total_phenols           178 non-null float64
Flavanoids              178 non-null float64
Nonflavanoid_phenols    178 non-null float64
Proanthocyanins         178 non-null float64
Color_intensity         178 non-null float64
Hue                     178 non-null float64
Diluted wines           178 non-null float64
Proline                 178 non-null int64
dtypes: float64(11), int64(3)
memory usage: 19.6 KB


In [7]:
wine.describe()

,Class,Alcohol,Malic_acid,Ash,Alcalinity_of_ash,Magnesium,Total_phenols,Flavanoids,Nonflavanoid_phenols,Proanthocyanins,Color_intensity,Hue,Diluted wines,Proline
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000
mean,1.938202,13.000618,2.336348,2.366517,19.494944,99.741573,2.295112,2.029270,0.361854,1.590899,5.058090,0.957449,2.611685,746.893258
std,0.775035,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474
min,1.000000,11.030000,0.740000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.410000,1.280000,0.480000,1.270000,278.000000
25%,1.000000,12.362500,1.602500,2.210000,17.200000,88.000000,1.742500,1.205000,0.270000,1.250000,3.220000,0.782500,1.937500,500.500000
50%,2.000000,13.050000,1.865000,2.360000,19.500000,98.000000,2.355000,2.135000,0.340000,1.555000,4.690000,0.965000,2.780000,673.500000
75%,3.000000,13.677500,3.082500,2.557500,21.500000,107.000000,2.800000,2.875000,0.437500,1.950000,6.200000,1.120000,3.170000,985.000000
max,3.000000,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.710000,4.000000,1680.000000


In [8]:
pd.isnull(wine).sum()

Class                   0
Alcohol                 0
Malic_acid              0
Ash                     0
Alcalinity_of_ash       0
Magnesium               0
Total_phenols           0
Flavanoids              0
Nonflavanoid_phenols    0
Proanthocyanins         0
Color_intensity         0
Hue                     0
Diluted wines           0
Proline                 0
dtype: int64

In [9]:
wine.head()

,Class,Alcohol,Malic_acid,Ash,Alcalinity_of_ash,Magnesium,Total_phenols,Flavanoids,Nonflavanoid_phenols,Proanthocyanins,Color_intensity,Hue,Diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [10]:
wine.Class.value_counts()  # value_counts()함수는 값을 출력하라는 함수.

2    71
1    59
3    48
Name: Class, dtype: int64

In [11]:
wine.Class #Class라는 속성변수를 method로 불러옴. wine["Class"] 와 같음.

0      1
1      1
2      1
3      1
4      1
      ..
173    3
174    3
175    3
176    3
177    3
Name: Class, Length: 178, dtype: int64

In [12]:
wine.Class = wine.Class-1 #0,1,2 로 만듦.

In [13]:
wineX = wine[col_name[1:]]

In [14]:
x_train, x_test, y_train, y_test = train_test_split(wineX, wine.Class, test_size=0.3, random_state=20200202)
print(x_train.shape)
print(y_train.shape)

#sklearn.model_selection에 있는 train_test_split 매서드를 사용

(124, 13)
(124,)


In [15]:
# MinMax 표준화
scaler = MinMaxScaler()
# sklearn.preprocessing.MinMaxScaler 임.
x_train = scaler.fit_transform(x_train) #Compute the minimum and maximum, then transform it.
x_test = scaler.transform(x_test)

#fit_transform 이나, fit 후 transform이나 같음. 또한 여기선 transform도 같음.

In [70]:
#===== 1개 은닉층을 갖는 신경망 모형의 은닉노드 개수 결정 (start)  =====#
# 즉, 모델 (model) selection을 하고 싶을 때, 아래의 코드를 보면됨. 이 작업은 시간이 굉장히 오래걸림.
# 참고: Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow, 2nd Edition

def build_model(n_neurons=30):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(n_neurons,activation="tanh",input_shape=[13])) 
    # 은닉층은 한개 , input 노드개수 13개: (,13) , output 노드개수는 n_neurons
    model.add(tf.keras.layers.Dense(3,activation="softmax"))
    model.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"]) 
    return model
# model 이 output

In [75]:
"""
참고: Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow, 2nd Edition

def build_model2(n_hidden=1,n_neurons=30,learning_rate=3e-3,input_shape=[13]):
    model=tf.keras.models.Sequential()
    options={"input_shape":input_shape}
    for layer in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons,activation="tanh",**options))
        options={}
    model.add(tf.keras.layers.Dense(3,activation="softmax"))
    optimizer=tf.keras.optimizers.SGD(learning_rate)
    model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer,metrics=["accuracy"]) 
    return model
"""

keras_class = tf.keras.wrappers.scikit_learn.KerasClassifier(build_model,epochs=100,verbose=0)
# grid_search를 하기 위해서 반드시 해야하는 작업. keras를 scikit_learn에사 사용할 수 있도록 함수를 감싸는 작업.

In [76]:
keras_class

In [77]:
para_distribs={
        "n_neurons":np.arange(1,41) # reciprocal 상호 연속적인 랜덤변수 또는 역수 
} 
# neuron의 수를 1부터 40까지 확인 // 사전형으로 정의해준다.

In [78]:
search_cv = RandomizedSearchCV(keras_class,para_distribs,n_iter=10,cv=3)

In [79]:
#sklearn 0.21에서는 아래가 돌아가나, 내가 가진 버전은 0.22 이다. 결국...미해결
search_cv.fit(x_train,y_train,
              validation_data=(x_test,y_test))


Train on 82 samples, validate on 54 samples
Epoch 1/100
82/82 [==============================] - 0s 4ms/sample - loss: 1.0607 - accuracy: 0.5000 - val_loss: 1.0554 - val_accuracy: 0.4444
Epoch 2/100
82/82 [==============================] - 0s 244us/sample - loss: 1.0416 - accuracy: 0.5122 - val_loss: 1.0399 - val_accuracy: 0.4630
Epoch 3/100
82/82 [==============================] - 0s 280us/sample - loss: 1.0219 - accuracy: 0.5366 - val_loss: 1.0249 - val_accuracy: 0.4815
Epoch 4/100
82/82 [==============================] - 0s 317us/sample - loss: 1.0069 - accuracy: 0.5854 - val_loss: 1.0112 - val_accuracy: 0.5000
Epoch 5/100
82/82 [==============================] - 0s 281us/sample - loss: 0.9906 - accuracy: 0.5732 - val_loss: 0.9974 - val_accuracy: 0.5185
Epoch 6/100
82/82 [==============================] - 0s 268us/sample - loss: 0.9754 - accuracy: 0.5976 - val_loss: 0.9834 - val_accuracy: 0.5556
Epoch 7/100
82/82 [==============================] - 0s 281us/sample - loss: 0.9624 - ac

83/83 [==============================] - 0s 337us/sample - loss: 0.8458 - accuracy: 0.7108 - val_loss: 0.8152 - val_accuracy: 0.7222
Epoch 12/100
83/83 [==============================] - 0s 301us/sample - loss: 0.8328 - accuracy: 0.7229 - val_loss: 0.7986 - val_accuracy: 0.7407
Epoch 13/100
83/83 [==============================] - 0s 253us/sample - loss: 0.8193 - accuracy: 0.7590 - val_loss: 0.7832 - val_accuracy: 0.8148
Epoch 14/100
83/83 [==============================] - 0s 241us/sample - loss: 0.8067 - accuracy: 0.7831 - val_loss: 0.7687 - val_accuracy: 0.8704
Epoch 15/100
83/83 [==============================] - 0s 253us/sample - loss: 0.7936 - accuracy: 0.8193 - val_loss: 0.7549 - val_accuracy: 0.9074
Epoch 16/100
83/83 [==============================] - 0s 241us/sample - loss: 0.7811 - accuracy: 0.8313 - val_loss: 0.7417 - val_accuracy: 0.9074
Epoch 17/100
83/83 [==============================] - 0s 229us/sample - loss: 0.7685 - accuracy: 0.8193 - val_loss: 0.7288 - val_accuracy

83/83 [==============================] - 0s 217us/sample - loss: 0.8590 - accuracy: 0.7831 - val_loss: 0.8465 - val_accuracy: 0.7963
Epoch 23/100
83/83 [==============================] - 0s 259us/sample - loss: 0.8474 - accuracy: 0.7952 - val_loss: 0.8343 - val_accuracy: 0.8148
Epoch 24/100
83/83 [==============================] - 0s 253us/sample - loss: 0.8360 - accuracy: 0.7952 - val_loss: 0.8221 - val_accuracy: 0.8519
Epoch 25/100
83/83 [==============================] - 0s 253us/sample - loss: 0.8250 - accuracy: 0.8193 - val_loss: 0.8098 - val_accuracy: 0.8704
Epoch 26/100
83/83 [==============================] - 0s 205us/sample - loss: 0.8136 - accuracy: 0.8313 - val_loss: 0.7977 - val_accuracy: 0.8889
Epoch 27/100
83/83 [==============================] - 0s 241us/sample - loss: 0.8027 - accuracy: 0.8434 - val_loss: 0.7855 - val_accuracy: 0.9074
Epoch 28/100
83/83 [==============================] - 0s 229us/sample - loss: 0.7916 - accuracy: 0.8795 - val_loss: 0.7736 - val_accuracy

82/82 [==============================] - 0s 256us/sample - loss: 0.6028 - accuracy: 0.9756 - val_loss: 0.6066 - val_accuracy: 0.9074
Epoch 34/100
82/82 [==============================] - 0s 244us/sample - loss: 0.5912 - accuracy: 0.9756 - val_loss: 0.5951 - val_accuracy: 0.9259
Epoch 35/100
82/82 [==============================] - 0s 268us/sample - loss: 0.5796 - accuracy: 0.9756 - val_loss: 0.5840 - val_accuracy: 0.9259
Epoch 36/100
82/82 [==============================] - 0s 244us/sample - loss: 0.5683 - accuracy: 0.9756 - val_loss: 0.5730 - val_accuracy: 0.9259
Epoch 37/100
82/82 [==============================] - 0s 256us/sample - loss: 0.5574 - accuracy: 0.9756 - val_loss: 0.5623 - val_accuracy: 0.9259
Epoch 38/100
82/82 [==============================] - 0s 256us/sample - loss: 0.5461 - accuracy: 0.9756 - val_loss: 0.5518 - val_accuracy: 0.9259
Epoch 39/100
82/82 [==============================] - 0s 256us/sample - loss: 0.5354 - accuracy: 0.9756 - val_loss: 0.5417 - val_accuracy

83/83 [==============================] - 0s 229us/sample - loss: 0.4092 - accuracy: 0.9277 - val_loss: 0.4157 - val_accuracy: 0.8889
Epoch 45/100
83/83 [==============================] - 0s 222us/sample - loss: 0.4012 - accuracy: 0.9277 - val_loss: 0.4073 - val_accuracy: 0.8889
Epoch 46/100
83/83 [==============================] - 0s 241us/sample - loss: 0.3928 - accuracy: 0.9518 - val_loss: 0.4002 - val_accuracy: 0.8889
Epoch 47/100
83/83 [==============================] - 0s 205us/sample - loss: 0.3849 - accuracy: 0.9398 - val_loss: 0.3932 - val_accuracy: 0.8889
Epoch 48/100
83/83 [==============================] - 0s 265us/sample - loss: 0.3771 - accuracy: 0.9639 - val_loss: 0.3858 - val_accuracy: 0.8889
Epoch 49/100
83/83 [==============================] - 0s 227us/sample - loss: 0.3701 - accuracy: 0.9639 - val_loss: 0.3779 - val_accuracy: 0.8889
Epoch 50/100
83/83 [==============================] - 0s 229us/sample - loss: 0.3624 - accuracy: 0.9639 - val_loss: 0.3716 - val_accuracy

...

83/83 [==============================] - 0s 410us/sample - loss: 0.3262 - accuracy: 0.9759 - val_loss: 0.2838 - val_accuracy: 0.9630
Epoch 63/100
83/83 [==============================] - 0s 244us/sample - loss: 0.3206 - accuracy: 0.9639 - val_loss: 0.2783 - val_accuracy: 0.9630
Epoch 64/100
83/83 [==============================] - 0s 253us/sample - loss: 0.3146 - accuracy: 0.9639 - val_loss: 0.2734 - val_accuracy: 0.9630
Epoch 65/100
83/83 [==============================] - 0s 205us/sample - loss: 0.3090 - accuracy: 0.9759 - val_loss: 0.2688 - val_accuracy: 0.9630
Epoch 66/100
83/83 [==============================] - 0s 205us/sample - loss: 0.3038 - accuracy: 0.9759 - val_loss: 0.2640 - val_accuracy: 0.9630
Epoch 67/100
83/83 [==============================] - 0s 217us/sample - loss: 0.2985 - accuracy: 0.9759 - val_loss: 0.2597 - val_accuracy: 0.9630
Epoch 68/100
83/83 [==============================] - 0s 205us/sample - loss: 0.2934 - accuracy: 0.9759 - val_loss: 0.2554 - val_accuracy

82/82 [==============================] - 0s 232us/sample - loss: 0.7518 - accuracy: 0.7805 - val_loss: 0.8035 - val_accuracy: 0.7222
Epoch 74/100
82/82 [==============================] - 0s 232us/sample - loss: 0.7470 - accuracy: 0.7805 - val_loss: 0.7984 - val_accuracy: 0.7222
Epoch 75/100
82/82 [==============================] - 0s 256us/sample - loss: 0.7422 - accuracy: 0.7805 - val_loss: 0.7933 - val_accuracy: 0.7407
Epoch 76/100
82/82 [==============================] - 0s 244us/sample - loss: 0.7374 - accuracy: 0.7805 - val_loss: 0.7885 - val_accuracy: 0.7593
Epoch 77/100
82/82 [==============================] - 0s 195us/sample - loss: 0.7326 - accuracy: 0.7805 - val_loss: 0.7838 - val_accuracy: 0.7593
Epoch 78/100
82/82 [==============================] - 0s 183us/sample - loss: 0.7278 - accuracy: 0.7805 - val_loss: 0.7790 - val_accuracy: 0.7593
Epoch 79/100
82/82 [==============================] - 0s 183us/sample - loss: 0.7229 - accuracy: 0.7927 - val_loss: 0.7740 - val_accuracy

83/83 [==============================] - 0s 241us/sample - loss: 0.7565 - accuracy: 0.8795 - val_loss: 0.7735 - val_accuracy: 0.8519
Epoch 85/100
83/83 [==============================] - 0s 169us/sample - loss: 0.7515 - accuracy: 0.8795 - val_loss: 0.7686 - val_accuracy: 0.8519
Epoch 86/100
83/83 [==============================] - 0s 205us/sample - loss: 0.7462 - accuracy: 0.8795 - val_loss: 0.7635 - val_accuracy: 0.8519
Epoch 87/100
83/83 [==============================] - 0s 217us/sample - loss: 0.7409 - accuracy: 0.8795 - val_loss: 0.7585 - val_accuracy: 0.8519
Epoch 88/100
83/83 [==============================] - 0s 241us/sample - loss: 0.7357 - accuracy: 0.8916 - val_loss: 0.7536 - val_accuracy: 0.8519
Epoch 89/100
83/83 [==============================] - 0s 205us/sample - loss: 0.7304 - accuracy: 0.8916 - val_loss: 0.7489 - val_accuracy: 0.8519
Epoch 90/100
83/83 [==============================] - 0s 181us/sample - loss: 0.7251 - accuracy: 0.8916 - val_loss: 0.7444 - val_accuracy

83/83 [==============================] - 0s 169us/sample - loss: 0.5437 - accuracy: 0.9398 - val_loss: 0.5017 - val_accuracy: 0.9444
Epoch 96/100
83/83 [==============================] - 0s 174us/sample - loss: 0.5384 - accuracy: 0.9398 - val_loss: 0.4967 - val_accuracy: 0.9444
Epoch 97/100
83/83 [==============================] - 0s 193us/sample - loss: 0.5330 - accuracy: 0.9398 - val_loss: 0.4916 - val_accuracy: 0.9444
Epoch 98/100
83/83 [==============================] - 0s 181us/sample - loss: 0.5278 - accuracy: 0.9518 - val_loss: 0.4866 - val_accuracy: 0.9444
Epoch 99/100
83/83 [==============================] - 0s 181us/sample - loss: 0.5228 - accuracy: 0.9518 - val_loss: 0.4817 - val_accuracy: 0.9444
Epoch 100/100
41/41 [==============================] - 0s 146us/sample - loss: 0.5008 - accuracy: 1.0000


RuntimeError: Cannot clone object <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x0000020D6604FF28>, as the constructor either does not set or modifies parameter n_neurons

In [53]:
print(search_cv.best_params_) 
print(search_cv.best_score_)

{'n_neurons': 21}
0.43263646960258484


In [54]:
model = search_cv.best_estimator_.model # save model

AttributeError: 'RandomizedSearchCV' object has no attribute 'best_estimator_'

In [ ]:
model.fit(x_train, y_train, epochs=100)

In [ ]:
result = model.evaluate(x_test,y_test)

print("loss:",result[0]) # 0.1158
print("accuracy:",result[1]) # 0.9815
#===== 1개 은닉층을 갖는 신경망 모형의 은닉노드 개수 결정 (end)  =====#

In [ ]:
#===== 위와 동일한 결과인지 확인하기 =====#
model_con = tf.keras.models.Sequential([
        tf.keras.layers.Dense(37,activation="tanh",input_shape=[13]),
        tf.keras.layers.Dense(3,activation="softmax")
        ])
    

In [ ]:
model_con.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model_con.summary()

In [ ]:
model_con.fit(x_train, y_train, epochs=100)

In [ ]:
model_con.evaluate(x_test,y_test)

In [93]:
#search_cv = RandomizedSearchCV(keras_class,para_distribs,n_iter=10,cv=3)
# hyper parameter의 공간이 커지면 RandomizedSearchCV를 쓰는게 좋음.
# 3 fold cv로 , n_iter은 parameters가 셈플링 되는 수. 1
# 우리는 40개중 10개만 셈플링해서 보는중.
# 자세한건 아래의 Reference 참고


################################ 아래는 미해결 ##########################
# 오류가 발생하는데, 해결을 못했다. 다만, 뭘 하려고하는지는 이해완료.

# search_cv.fit(x_train,y_train,epochs=100,
#               validation_data=(x_test,y_test),
#               callbacks=[tf.keras.callbacks.EarlyStopping(patience=10)])
# 
# search_cv.best_params_ 
# search_cv.best_score_ 
# model = search_cv.best_estimator_.model # save model
# 
# model.fit(x_train, y_train, epochs=100)
# result = model.evaluate(x_test,y_test)
# 
# print("loss:",result[0]) # 0.1158
# print("accuracy:",result[1]) # 0.9815
# #===== 1개 은닉층을 갖는 신경망 모형의 은닉노드 개수 결정 (end)  =====#
# 
# #===== 위와 동일한 결과인지 확인하기 =====#
# model_con = tf.keras.models.Sequential([
#         tf.keras.layers.Dense(37,activation="tanh",input_shape=[13]),
#         tf.keras.layers.Dense(3,activation="softmax")
#         ])
#     
# model_con.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])
# model_con.summary()
# model_con.fit(x_train, y_train, epochs=100)
# model_con.evaluate(x_test,y_test)

## 2. Example 1-3
---

In [66]:
"""
참고교재: 텐서플로로 배우는 딥러닝 (박혜정, 석경하, 심주용, 황창하 공저)

예제 1-3. UCI 기계학습 저장소에 있는 Boston 집값 데이터를 활용하여 
집값(MEDV)을 출력변수로 하는 신경망 회귀모형을 생성하여 검증용 자료에 회귀모형의 
성능을 평가하시오. 평가는 housing 데이터 중 훈련자료 70%와 검정자료 30%로 
분할하여 검정하시오(random.state=200으로 고정).
"""

# 변수명 설정
boston_name = ["CRIM","ZN","INDUS",
          "CHAS","NOX","RM","AGE","DIS",
          "RAD","TAX","PTRATIO",    
          "B", "LSTAT", "MEDV"]

In [132]:

# 데이터 불러오기
boston = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data",
     header=None, sep="\s+", names=boston_name, na_values=["NA", "null", ""])
# "\s+" 는 정규표현식으로, 공백이 하나 이상인 경우를 구분자로 지으라는 뜻.

In [133]:
boston.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
CRIM       506 non-null float64
ZN         506 non-null float64
INDUS      506 non-null float64
CHAS       506 non-null int64
NOX        506 non-null float64
RM         506 non-null float64
AGE        506 non-null float64
DIS        506 non-null float64
RAD        506 non-null int64
TAX        506 non-null float64
PTRATIO    506 non-null float64
B          506 non-null float64
LSTAT      506 non-null float64
MEDV       506 non-null float64
dtypes: float64(12), int64(2)
memory usage: 55.5 KB


In [134]:
boston.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677082,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


In [136]:
# 결측값 제거
pd.isnull(boston).sum()
boston1 = boston.dropna()
boston1.dtypes

CRIM       float64
ZN         float64
INDUS      float64
CHAS         int64
NOX        float64
RM         float64
AGE        float64
DIS        float64
RAD          int64
TAX        float64
PTRATIO    float64
B          float64
LSTAT      float64
MEDV       float64
dtype: object

In [137]:
# 종속변수 설정
y4 = boston1["MEDV"]

In [138]:
x_train4, x_test4, y_train4, y_test4 = train_test_split(boston1[boston_name[0:-1]], y4, test_size=0.3, random_state=200)
# boston_name[0:-1] 은 0부터 -1번째 전까지 불러오라는뜻. -1번째란, 뒤에서 첫번째를 뜻함.
# 즉, 0이상 -1번째 미만.

In [140]:
minmax4 = MinMaxScaler()
x_train4 = minmax4.fit_transform(x_train4)
x_test4 = minmax4.transform(x_test4) 

In [141]:
def build_model4(n_hidden=1,n_neurons=30,input_shape=[x_train4.shape[1]]):
    model = tf.keras.models.Sequential()
    options = {"input_shape":input_shape}
    for layer in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons,activation="relu",**options))
        options={}
    model.add(tf.keras.layers.Dense(1))
    model.compile(loss="mse",optimizer=tf.keras.optimizers.RMSprop(0.001),metrics=["mae","mse"])
    return model

# **options 는 키워드 파라미터이다. 
# 위에서는 후에 options={}로 초기화 시켜서 사용했다.
# 이런것도 있다고 알아두기.
# 측도로 MAE 와 MSE를 사용하였으며, Adam에서 RMSprop를 사용함을 확인하기.
# loss로도 MSE를 사용.

#아래의 Reference 를 참고하면 더 자세히 볼 수 있다.

In [142]:
para4={
      "n_hidden":np.arange(1,10),
      "n_neurons":np.arange(1,100)
      }

#조합은 천개.

In [143]:
keras_class4 = tf.keras.wrappers.scikit_learn.KerasRegressor(build_model4)

In [144]:
search_cv4_1 = RandomizedSearchCV(keras_class4,para4,n_iter=10,cv=3)

In [ ]:
#역시나, 이후의 작업들이 되지 않는다... 

# search_cv4_1.fit(x_train4,y_train4,epochs=100,
#               validation_data=(x_test4,y_test4),
#               callbacks=[tf.keras.callbacks.EarlyStopping(patience=10)])

# search_cv4_1.best_params_ # 5hidden 70 neuron
# search_cv4_1.best_score_
# model4_1 = search_cv4_1.best_estimator_.model

# # 검정용 데이터 적용
# pred4_1 = model4_1.predict(x_test4)
# result4_1 = model4_1.evaluate(x_test4, y_test4)
# print("MSE:",result4_1[0]) # 18.88
# print("MAE:",result4_1[1]) # 3.11


## 한 번 더  
## randomized이기 때문에 돌릴때마다 달라짐. 일부의 조합만 봤기 때문.
## 이 경우 10개의 조합만 살펴본 셈이다.

# search_cv4_2 = RandomizedSearchCV(keras_class4,para4,n_iter=10,cv=3)
# search_cv4_2.fit(x_train4,y_train4,epochs=100,
#               validation_data=(x_test4,y_test4),
#               callbacks=[tf.keras.callbacks.EarlyStopping(patience=10)])

# search_cv4_2.best_params_ # 6hidden 36 neuron
# model4_2 = search_cv4_2.best_estimator_.model

# # 검정용 데이터 적용
# pred4_2 = model4_2.predict(x_test4)
# result4_2 = model4_2.evaluate(x_test4, y_test4)
# print("MSE:",result4_2[0])
# print("MAE:",result4_2[1])

#달라진 결과값 확인.

## 3. Reference
---

- [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)
- [Keyword Parameter](https://wikidocs.net/24)